In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import matplotlib.patheffects as path_effects
import geopy.distance

In [1]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 234.8/234.8 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.1/312.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 428.0/428.0 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.2/148.2 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 17.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.6/64.6 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 MB 10.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 81.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 80.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [10]:
!wget https://lodmedia.hb.bizmrg.com/case_files/1113064/test_dataset_test_data_Mediawise.zip
!unzip /content/test_dataset_test_data_Mediawise.zip

--2024-07-07 05:32:41--  https://lodmedia.hb.bizmrg.com/case_files/1113064/test_dataset_test_data_Mediawise.zip
Resolving lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)... 95.163.53.117
Connecting to lodmedia.hb.bizmrg.com (lodmedia.hb.bizmrg.com)|95.163.53.117|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 266963 (261K) [application/x-zip-compressed]
Saving to: ‘test_dataset_test_data_Mediawise.zip.1’

test_dataset_test_d 100%[===================>] 260.71K   303KB/s    in 0.9s    

2024-07-07 05:32:43 (303 KB/s) - ‘test_dataset_test_data_Mediawise.zip.1’ saved [266963/266963]

Archive:  /content/test_dataset_test_data_Mediawise.zip
   creating: test_data_Mediawise/
  inflating: test_data_Mediawise/test_data.json  
  inflating: test_data_Mediawise/.DS_Store  
  inflating: __MACOSX/test_data_Mediawise/._.DS_Store  
  inflating: test_data_Mediawise/sample_submission.csv  


In [11]:
df=pd.read_json('/content/test_data_Mediawise/test_data.json')

In [12]:
df=pd.concat([df,pd.json_normalize(df['targetAudience'])], axis=1)
#%%
df=df.drop(['targetAudience','id'], axis=1)

In [13]:
def mean_distance(points):  # среднее растояние от центра Москвы до точек
    count = 0
    coords = 55.753567, 37.621409
    sm = 0
    for i in points:
        count += 1
        new_coords = i['lat'], i['lon']
        sm += geopy.distance.geodesic(coords, new_coords).km
    return sm / count

In [14]:
df['mean_distance'] = df.points.apply(mean_distance)


In [15]:
#Делит отрезок на равные интервалы
def split_on_intervals(min_val, max_val, n):
  step = (max_val - min_val)/n
  intervals = [min_val+(step*x) for x in range(n+1)]
  return intervals

#Создает регионы для поля
def create_groups(x_intervals, y_intervals):
  groups = {}
  x_intervals = np.concatenate([[-np.inf], x_intervals, [np.inf]])
  y_intervals = np.concatenate([[-np.inf], y_intervals, [np.inf]])

  for x_i in range(len(x_intervals)-1):
    for y_i in range(len(y_intervals)-1):
      groups[f'x : {x_intervals[x_i]} - {x_intervals[x_i+1]} | y : {y_intervals[y_i]} - {y_intervals[y_i+1]}'] = 0

  return groups
#%%
#Сортирует точки по регионам
def sort_on_groups(x_vals, y_vals, x_intervals, y_intervals, groups, only_vals = False):

  for x, y in zip(x_vals, y_vals):
    for x_i in range(len(x_intervals)-1):
      for y_i in range(len(y_intervals)-1):
        if ((x_intervals[x_i] <= x < x_intervals[x_i+1]) and (y_intervals[y_i] <= y < y_intervals[y_i+1])):
          groups[f'x : {x_intervals[x_i]} - {x_intervals[x_i+1]} | y : {y_intervals[y_i]} - {y_intervals[y_i+1]}'] += 1


  if only_vals:
    return list(groups.values())

  return groups
#%%
def create_dataset(config, df):
  x_intervals = split_on_intervals(config['min_xval'], config['max_xval'], config['x_ngroups'])
  y_intervals = split_on_intervals(config['min_yval'], config['max_yval'], config['y_ngroups'])

  groups = create_groups(x_intervals, y_intervals)

  groups_values=[]

  for i in range(len(df)):
    g = df.iloc[i]
    points = np.array([[float(x['lat']), float(x['lon'])] for x in g['points']])


    group_values = sort_on_groups(points[:,0], points[:,1], x_intervals, y_intervals, groups.copy(), only_vals = True)
    groups_values.append(group_values)


  groups_values = np.array(groups_values)

  for i in range(len(groups.keys())):
    groups[list(groups.keys())[i]]=groups_values[:,i]


  return groups


In [16]:
config = {'min_xval':55.55, 'max_xval':55.95, 'min_yval':37.3, 'max_yval':37.9, 'x_ngroups': 50, 'y_ngroups': 50}


In [17]:
dataset = pd.DataFrame(create_dataset(config, df))


In [19]:
X = dataset.copy()
X = X.set_index(df.index)
X['gender'] = df['gender']
X['ageFrom'] = df['ageFrom']
X['ageTo'] = df['ageTo']
X['income'] = df['income']
X['mean_distance'] = df['mean_distance']

In [8]:
predictor = TabularPredictor.load('/content/drive/MyDrive/hacks_ai/with_nn_3')

In [20]:
pred = predictor.predict(X)

In [29]:
pred

0       0.747751
1      13.125265
2      92.860382
3      89.586273
4       1.934547
         ...    
383     0.638640
384    32.241699
385    19.171576
386    28.506176
387     0.587850
Name: value, Length: 388, dtype: float32

In [26]:
sample = pd.read_csv('/content/test_data_Mediawise/sample_submission.csv')

In [30]:
sample.value = pred

In [31]:
sample

,hash,value
0,3a94fc3e5c9d8d72,0.747751
1,bcc4e68cbec07f33,13.125265
2,d8da298a5c2b2f27,92.860382
3,5c75b1a1602e33d7,89.586273
4,609bcaed570f5815,1.934547
...,...,...
383,4e154970e1c6ae2b,0.638640
384,3c15071c77f49893,32.241699
385,c4576f9dcda42732,19.171576
386,c9c9b7cbe3a925d4,28.506176


In [32]:
sample.to_csv('submission2ag_SPA.csv', index=False)